### Markdown 说明文

#### 代码功能概述
该代码旨在通过最优化技术找到给定函数 `f(x) = a + b * cos(x) + c * sin(x)` 的最优 `x` 值，函数的系数 `a`, `b`, 和 `c` 在某个范围内有不确定性（由 `deltas` 确定的盒型约束）。通过 `scipy.optimize.minimize` 函数，代码最小化目标函数，找到使 `f(x)` 达到最优的 `x`。

#### 主要步骤

1. **随机生成参数**：  
   使用 `np.random.uniform` 随机生成系数 `hat_z = [a, b, c]`，并生成不确定性范围 `deltas`。  
   - `hat_z` 在 [1.0, 3.0] 之间随机生成。
   - `deltas` 在 [0.01, 0.1] 之间随机生成。

2. **定义优化目标函数**：  
   `inner_ojbective(x, deltas)` 函数通过给定的 `x`，计算函数 `f(x)`。该函数使用 `hat_z` 和 `deltas` 通过盒型约束来确定每个参数的上下界。

3. **最优化**：  
   使用 `scipy.optimize.minimize` 对 `x` 进行最小化，寻找使得 `f(x)` 最优的 `x`。初始猜测 `x0 = 0`，可以选择加上边界条件 `[0, 2*np.pi]` 限制 `x` 的取值范围。

4. **输出结果**：  
   打印最优的 `x` 值和相应的目标函数值。

#### 备注
- 可以通过调整初始猜测值 `x0` 及约束范围（使用 `bounds` 参数）来影响优化结果。


In [17]:
import numpy as np
from scipy.optimize import minimize

# 定义函数 f(x) = a + b * cos(x) + c * sin(x)
def f(x, a, b, c):
    return a + b * np.cos(x) + c * np.sin(x)

# 随机生成 hat_z 和 deltas
hat_z = np.random.uniform(1.0, 3.0, size=3)
deltas = np.random.uniform(0.01, 0.1, size=3)

# 定义参数的不确定性集合
def inner_ojbective(x, deltas):
    z_star = np.zeros(len(deltas))
    l = hat_z - deltas
    u = hat_z + deltas

    # 确保 x 是标量，因为 minimize 会传递数组
    if isinstance(x, np.ndarray):
        x = x[0]

    tx = np.array([1, np.cos(x), np.sin(x)])
    for i in range(len(tx)):
        if tx[i] > 0:
            z_star[i] = u[i]
        elif tx[i] < 0:
            z_star[i] = l[i]
        else:
            z_star[i] = u[i]
    
    return f(x, z_star[0], z_star[1], z_star[2])

# 对 x 进行优化的初始猜测
x0 = 0  # 确保 x0 是一个数组

# 使用 scipy 的 minimize 来最小化 f(x) 对 x
result = minimize(lambda x: inner_ojbective(x, deltas), x0)  
# result = minimize(lambda x: inner_ojbective(x, deltas), x0, bounds=[(0, 2*np.pi)])  

print("最优 x:", result.x)
print("最优目标函数值:", result.fun)


最优 x: [-2.34195043]
最优目标函数值: 0.7946975613989358


In [18]:
import numpy as np
from scipy.optimize import minimize

# 定义函数 f(x) = a + b * cos(x) + c * sin(x)
def f(x, a, b, c):
    return a + b * np.cos(x) + c * np.sin(x)

# 随机生成 hat_z 和 deltas
hat_z = np.random.uniform(1.0, 3.0, size=3)
deltas = np.random.uniform(0.01, 0.1, size=3)

# np.where(condition, x, y)
# np.where 是 NumPy 提供的一个函数，用于根据条件选择数组中的元素。它的功能类似于三元表达式 x if condition else y，但可以对数组的每个元素进行操作。

# 定义目标函数
def objective(x):
    x = x[0] if isinstance(x, np.ndarray) else x
    tx = np.array([1, np.cos(x), np.sin(x)])
    z_star = np.where(tx > 0, hat_z + deltas, hat_z - deltas)
    return f(x, *z_star)

# 优化
result = minimize(objective, 0)
print("最优 x:", result.x)
print("最优目标函数值:", result.fun)


最优 x: [-2.50554907]
最优目标函数值: 0.6567378811783605
